# Lab03
## BLAST steps
This notebook provides a simplified ground-level look at how BLASTP (Basic Local Alignment Search Tool for Proteins) functions.

For the Activity 3:
1) Run all the code. Make sure it works and you get the outputs
2) Answer the question (x3) along the notebook
3) Save the notebook with your name (not initials). Save as HTML and export as PDF. Then, submit the assignment to MyCourses.

### Our sample sequences (Amino Acids)

In [ ]:

query = "HEAGAWGHEE"
database = "PAWHEAELLGHEAGAWGHPA"
word_size = 3 

print(f"Query: {query}")
print(f"Database: {database}")

### Seeding (Word Generation & Indexing)

The algorithm starts by breaking the query sequence into small "words" of a fixed size (here, k=3).

In [ ]:
def index_query(sequence, w):
    words = {}
    for i in range(len(sequence) - w + 1):
        word = sequence[i:i+w]
        if word not in words:
            words[word] = []
        words[word].append(i)
    return words

query_words = index_query(query, word_size)
print(f"Query words (Size {word_size}) and their indices:")
print(query_words)

### Scanning the Database

The find_word_hits function scans the database for any occurrences of the words in your index. These matches are called seeds. This is a "hit/no-hit" phase that avoids expensive alignments.

In [ ]:
def find_word_hits(db, query_index, w):
    hits = []
    for i in range(len(db) - w + 1):
        db_word = db[i:i+w]
        if db_word in query_index:
            # A word might appear multiple times in the query
            for query_pos in query_index[db_word]:
                hits.append({
                    'word': db_word,
                    'query_start': query_pos,
                    'db_start': i
                })
    return hits

hits = find_word_hits(database, query_words, word_size)

# Visualizing the hits using a DataFrame
import pandas as pd
df_hits = pd.DataFrame(hits)
print("\nWord Matches (Seeds) Found:")
print(df_hits)

### Scoring system

A simplified mini-BLOSUM62 for our example

In [ ]:
# A simplified mini-BLOSUM62 for our example
# (In a real scenario, you'd use a library like Bio.Align.substitution_matrices)
scoring_matrix = {
    # --- A (Alanine) ---
    ('A', 'A'):  4, ('A', 'G'):  0, ('A', 'W'): -4, ('A', 'H'): -2, ('A', 'E'): -1,
    
    # --- G (Glycine) ---
    ('G', 'A'):  0, ('G', 'G'):  4, ('G', 'W'): -4, ('G', 'H'): -2, ('G', 'E'): -1,
    
    # --- W (Tryptophan) ---
    ('W', 'A'): -4, ('W', 'G'): -4, ('W', 'W'): 11, ('W', 'H'): -3, ('W', 'E'): -5,
    
    # --- H (Histidine) ---
    ('H', 'A'): -2, ('H', 'G'): -2, ('H', 'W'): -3, ('H', 'H'):  8, ('H', 'E'):  0,
    
    # --- E (Glutamate) ---
    ('E', 'A'): -1, ('E', 'G'): -1, ('E', 'W'): -5, ('E', 'H'):  0, ('E', 'E'):  5
}

def score_words(word1, word2):
    score = 0
    for a, b in zip(word1, word2):
        score += scoring_matrix.get((a, b), -4) # Default to -4 for unknown pairs
    return score

### Stablish Neighborhood table

The get_neighborhood function is the "secret sauce" that makes BLAST a heuristic (approximate) search rather than a slow, literal search.

In [ ]:
import itertools

def get_neighborhood(query_word, threshold, alphabet):
    neighbors = []
    # Generate all possible words of the same length
    for permutation in itertools.product(alphabet, repeat=len(query_word)):
        candidate = "".join(permutation)
        if score_words(query_word, candidate) >= threshold:
            neighbors.append(candidate)
    return neighbors

# Example: Finding neighbors for the word "HEA"
alphabet = "AGWHE" # Simplified for this demo
word = "HEA"
threshold = 11

neighborhood = get_neighborhood(word, threshold, alphabet)
print(f"Neighbors for '{word}' with threshold {threshold}:")
print(neighborhood)

### Maps Neighbors to Positions

The build_neighborhood_index function is the step that compiles all the "neighbor"  words into a searchable index for the entire query sequence.

In [ ]:
def build_neighborhood_index(query, word_size, threshold, alphabet):
    index = {}
    for i in range(len(query) - word_size + 1):
        actual_word = query[i:i+word_size]
        neighbors = get_neighborhood(actual_word, threshold, alphabet)
        
        for neighbor in neighbors:
            if neighbor not in index:
                index[neighbor] = []
            index[neighbor].append(i)
    return index

# Build the index with neighborhood logic
neighborhood_index = build_neighborhood_index(query, word_size, 11, "AGWHE")

# Re-run the scan on the database
new_hits = find_word_hits(database, neighborhood_index, word_size)

print(f"\nFound {len(new_hits)} hits using neighborhood logic (vs earlier exact matches).")
pd.DataFrame(new_hits).head()

### Extension (HSP Generation)

Once a seed is found, the extend_hit function tries to grow the alignment in both directions (left and right).

In [ ]:
def extend_hit(query, db, q_start, db_start, word_size, matrix, drop_off):
    # Initial score of the word match
    current_score = score_words(query[q_start:q_start+word_size], 
                                db[db_start:db_start+word_size])
    
    max_score = current_score
    
    # Coordinates of the alignment
    q_ext_start, q_ext_end = q_start, q_start + word_size
    db_ext_start, db_ext_end = db_start, db_start + word_size
    
    # 1. Extend Right
    i = 1
    while (q_start + word_size + i <= len(query) and 
           db_start + word_size + i <= len(db)):
        char_q = query[q_start + word_size + i - 1]
        char_db = db[db_start + word_size + i - 1]
        current_score += matrix.get((char_q, char_db), -4)
        
        if current_score > max_score:
            max_score = current_score
            q_ext_end = q_start + word_size + i
            db_ext_end = db_start + word_size + i
        
        if current_score < (max_score - drop_off):
            break
        i += 1

    # 2. Extend Left (reset current_score to max_score achieved during right extension)
    current_score = max_score
    i = 1
    while (q_start - i >= 0 and db_start - i >= 0):
        char_q = query[q_start - i]
        char_db = db[db_start - i]
        current_score += matrix.get((char_q, char_db), -4)
        
        if current_score > max_score:
            max_score = current_score
            q_ext_start = q_start - i
            db_ext_start = db_start - i
            
        if current_score < (max_score - drop_off):
            break
        i += 1
        
    return {
        'score': max_score,
        'query_seq': query[q_ext_start:q_ext_end],
        'db_seq': db[db_ext_start:db_ext_end],
        'query_range': (q_ext_start, q_ext_end),
        'db_range': (db_ext_start, db_ext_end)
    }

### Results

In [ ]:
all_hsps = []
drop_off = 2

for hit in new_hits:
    hsp = extend_hit(query, database, hit['query_start'], hit['db_start'], word_size, scoring_matrix, drop_off)
    all_hsps.append(hsp)

# Filter for unique HSPs and sort by score
hsps_df = pd.DataFrame(all_hsps).drop_duplicates(subset=['query_seq', 'db_seq'])
hsps_df = hsps_df.sort_values(by='score', ascending=False)

print("High-scoring Segment Pairs (HSPs):")
hsps_df

In [ ]:
def display_hsp(hsp):
    # Extract data
    q_seq = hsp['query_seq']
    db_seq = hsp['db_seq']
    q_start, q_end = hsp['query_range']
    db_start, db_end = hsp['db_range']
    
    # Create the middle "match" line
    # In BLAST, '|' is used for identity, '+' for similar, ' ' for mismatch
    match_line = ""
    for q_char, db_char in zip(q_seq, db_seq):
        if q_char == db_char:
            match_line += "|"
        elif scoring_matrix.get((q_char, db_char), -4) > 0:
            match_line += "+"
        else:
            match_line += " "

    # Format with consistent widths for coordinates (e.g., 5 spaces)
    print(f"Score: {hsp['score']}")
    print(f"Query  {q_start:<5} {q_seq} {q_end:>5}")
    print(f"       {' ' * 5} {match_line}")
    print(f"Sbjct  {db_start:<5} {db_seq} {db_end:>5}\n")

# Re-run the display
for _, row in hsps_df.head(2).iterrows():
    display_hsp(row)

# <font color=red> Activity 3 </font>

Now that we have found our High-scoring Segment Pairs (HSPs), we need to know: Is this match actually interesting, or could it have happened by random chance?

Complete the function below using the standard BLAST formula. Feel free to use the "function" template below or create a new code to implement the E-value formula in the results.

In [ ]:
import math

def calculate_e_value(score, query_len, db_len):
    """
    Calculates the E-value (Expect value) for a given alignment.
    """
    # Constants for BLOSUM62 / Word-size 3
    # These are typically pre-calculated by NCBI
    Lambda = 0.267
    K = 0.041
    
    # --- STUDENT TASK: IMPLEMENT THE FORMULA BELOW ---
    # m = query_len
    # n = db_len
    # S = score
    # Hint: Use math.exp() for the exponential part
    
    e_value = # ... [check your lecture slides for the formula] ...
    
    return e_value

# Apply the calculation to our results table
query_len = len(query)
db_len = len(database)

# Use a lambda function to apply your calculation to every row in the dataframe
hsps_df['E-value'] = hsps_df['score'].apply(lambda s: calculate_e_value(s, query_len, db_len))

print("Final BLAST Results with E-values:")
# Displaying the most important columns
display(hsps_df[['score', 'E-value', 'query_seq', 'db_seq']].sort_values(by='score', ascending=False))

### Full HSP Extension Plot

This code will capture the score progression in both directions and center the "seed" at the middle of the graph.

The Orange Line: Represents the original 3-letter word (seed) found during the scanning phase.

The Green line: Shows the "High-scoring Segment Pair" (HSP). If the sequences are highly similar, this peak will stay high for a long time.

The Red Dotted Line: The "Drop-off" boundary. This is the mathematical "patience" of the algorithm; once the teal line stays below this red line, the search for this specific segment stops.

In [ ]:
import matplotlib.pyplot as plt

def run_and_plot_blast_full(query, database, word_size=3, drop_off=5):
    # 1. Get the first hit
    query_index = index_query(query, word_size)
    hits = find_word_hits(database, query_index, word_size)
    
    if not hits:
        print("No hits found!")
        return
    
    hit = hits[0]
    q_start, db_start = hit['query_start'], hit['db_start']
    
    # Initial seed score
    seed_score = score_words(query[q_start:q_start+word_size], database[db_start:db_start+word_size])
    
    # 2. Extend Right
    right_scores = []
    current_score = seed_score
    max_score_right = seed_score
    i = 1
    while (q_start + word_size + i <= len(query) and db_start + word_size + i <= len(database)):
        char_q = query[q_start + word_size + i - 1]
        char_db = database[db_start + word_size + i - 1]
        current_score += scoring_matrix.get((char_q, char_db), -4)
        right_scores.append(current_score)
        if current_score > max_score_right: max_score_right = current_score
        if current_score < (max_score_right - drop_off): break
        i += 1

    # 3. Extend Left
    left_scores = []
    current_score = seed_score
    max_score_left = seed_score
    i = 1
    while (q_start - i >= 0 and db_start - i >= 0):
        char_q = query[q_start - i]
        char_db = database[db_start - i]
        current_score += scoring_matrix.get((char_q, char_db), -4)
        left_scores.insert(0, current_score) # Insert at start to keep sequence order
        if current_score > max_score_left: max_score_left = current_score
        if current_score < (max_score_left - drop_off): break
        i += 1

    # Combine: Left scores + Seed score + Right scores
    full_scores = left_scores + [seed_score] + right_scores
    seed_index = len(left_scores)
    
    # 4. Plotting
    plt.figure(figsize=(10, 5))
    plt.plot(full_scores, marker='o', linestyle='-', color='green', label='Alignment Score')
    plt.axvline(x=seed_index, color='orange', linestyle='--', label='Seed Position')
    
    # Threshold line based on the global maximum
    global_max = max(full_scores)
    plt.axhline(y=global_max - drop_off, color='red', linestyle=':', label='Termination Threshold')
    
    plt.title(f"Full BLAST Extension: '{query[q_start:q_start+word_size]}' at Query Index {q_start}")
    plt.xlabel("Relative Alignment Position")
    plt.ylabel("Score")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()

run_and_plot_blast_full(query, database)

# <font color=red> Activity 3 </font>
Run the notebook multiple times while varying one parameter at a time:

- Word size (word_size)
- Threshold (threshold)

Explain (and document) how changes in each parameter affect the balance between search speed and biological sensitivity, and how this is reflected in the alignments obtained.

## Answer
Type your answer (text) here.

# <font color=red> Activity 3 </font>
The "X-Factor" (Drop-off Threshold)
You are comparing two sequences that start identically but diverge significantly in the middle before matching again at the very end:

    Query: HHHAGWWHHH

    Database: HHHAGAAAHH

Learn how the Drop-off setting acts as the algorithm's "patience." The Drop-off value is a "cutoff" point. It tells BLAST: "If the score falls too far below the highest peak we found, stop searching."

Using the cell code below, find two values for Drop-off (drop_off = X) settings in the algorithm where:
 
    -Finds a match only with the first region
    -Extends to the full sequence

And plot the respective HSP extensions.

Hint: WW (Tryptophans) in the query are replaced by AA (Alanines) in the database. According to our scoring_matrix, this mismatch is a heavy penalty (-4 points per letter).

In [ ]:
# 1. Define the sequences
query_seq = "HHHAGWWHHH"
db_seq    = "HHHAGAAAHH"

# 2. Run the experiment
print("--- TEST A: Low Patience (drop_off = X) ---")
run_and_plot_blast_full(query_seq, db_seq, word_size=3, drop_off=X)

print("--- TEST B: High Patience (drop_off = X) ---")
run_and_plot_blast_full(query_seq, db_seq, word_size=3, drop_off=X)